In [1]:
%load_ext pipescript

In [2]:
inp = """
aaa: you hhh
you: bbb ccc
bbb: ddd eee
ccc: ddd eee fff
ddd: ggg
eee: out
fff: out
ggg: out
hhh: ccc fff iii
iii: out
""".strip().splitlines()

In [3]:
with open("11.txt", "r") as f:
    inp = f.read().strip().splitlines()

In [4]:
inp = inp |> map[$.strip().split(": ") *|> ($, $.split())]

In [5]:
graph = {}
for node, adjs in inp:
    for adj in adjs:
        graph.setdefault(adj, set()).add(node)

In [6]:
def get_npaths(from_node, to_node):
    seen = set()
    n_neighbors_reachable_from = {}
    def dfs(node):
        n_neighbors_reachable_from[node] = n_neighbors_reachable_from.get(node, 0) + 1
        if node in seen:
            return
        seen.add(node)
        for adj in graph.get(node, []):
            dfs(adj)
    dfs(to_node)
    
    weight = {}
    neighbors_seen = {}
    weight[to_node] = 1
    q = [to_node]
    while q:
        node = q.pop(0)
        for adj in graph.get(node, []):
            weight[adj] = weight.get(adj, 0) + weight[node]
            neighbors_seen[adj] = neighbors_seen.get(adj, 0) + 1
            if neighbors_seen[adj] == n_neighbors_reachable_from[adj]:
                q.append(adj)
    return weight.get(from_node, 0)

In [7]:
get_npaths("you", "out")

658

In [8]:
from itertools import permutations

def get_npaths_with_intermediates(start, end, intermediates):
    ret = 0
    for perm in permutations(intermediates):
        n_paths_with_order = 1
        for from_node, to_node in zip((start,) + perm, perm + (end,)):
            n_paths_with_order *= get_npaths(from_node, to_node)
            if n_paths_with_order == 0:
                break
        ret += n_paths_with_order
    return ret

In [9]:
get_npaths_with_intermediates("svr", "out", ["dac", "fft"])

371113003846800